<a href="https://colab.research.google.com/github/MehmetKorkut/Spotify_Dashboard/blob/main/Spotify_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>

# ***Visualizing Spotify Data with Python and Tableau***

![picture](https://wallpaperaccess.com/full/667773.jpg)

</center>

The code source in this project is from https://towardsdatascience.com/visualizing-spotify-data-with-python-tableau-687f2f528cdd. It belongs to Anne Bode. I only made a few changes because of the problems occured due to my data. The main reason why I did this is that I think it is fun to see my music data visually and learning tableau at the same time.

In [ ]:
#packages
import pandas as pd
import numpy as np
import requests

In [ ]:
#creating dataset
# read your 1+ StreamingHistory files (depending on how extensive your streaming history is) into pandas dataframes
df_stream0 = pd.read_json('StreamingHistory0.json')
df_stream1 = pd.read_json('StreamingHistory1.json')
df_stream2 = pd.read_json('StreamingHistory2.json')

# merge streaming dataframes
df_stream = pd.concat([df_stream0, df_stream1,df_stream2])

# create a 'UniqueID' for each song by combining the fields 'artistName' and 'trackName'
df_stream['UniqueID'] = df_stream['artistName'] + ":" + df_stream['trackName']

df_stream.head()

,endTime,artistName,trackName,msPlayed,UniqueID
0,2020-09-21 07:39,Erik Eriksson,White Noise 3 Hour Long,2926566,Erik Eriksson:White Noise 3 Hour Long
1,2020-09-21 15:03,Ini Kamoze,WORLD-A-REGGAE Music,35390,Ini Kamoze:WORLD-A-REGGAE Music
2,2020-09-21 17:40,Exodus,Salt the Wound,221866,Exodus:Salt the Wound
3,2020-09-22 06:32,Erik Eriksson,White Noise 3 Hour Long,9967424,Erik Eriksson:White Noise 3 Hour Long
4,2020-09-22 07:48,Megadeth,Dystopia,183490,Megadeth:Dystopia


In [ ]:
df_library=pd.read_csv('YourLibrary.csv')
# add UniqueID column (same as above)
df_library['UniqueID'] = df_library['Artist'] + ":" + df_library['Track']

# add column with track URI stripped of 'spotify:track:'
new = df_library["Uri"].str.split(":", expand = True)
df_library['Track_Uri'] = new[2]
df_library.drop(columns=['Document Index (generated)','tracks. Index (generated)','Number of Records per YourLibrary.json'],inplace=True)
df_library.head()

,Album,Artist,Track,Uri,UniqueID,Track_Uri
0,Master Of Puppets,Metallica,"Master Of Puppets - Live At The Country Club, ...",spotify:track:7FDuSqP0qxvFFrcftd93aB,Metallica:Master Of Puppets - Live At The Coun...,7FDuSqP0qxvFFrcftd93aB
1,Reunion,Black Sabbath,N.I.B. - Live,spotify:track:5uKsIijGkAbWmYAEdnCPhG,Black Sabbath:N.I.B. - Live,5uKsIijGkAbWmYAEdnCPhG
2,Live MMXIV,Pentagram,"Apokalips - Garajistanbul, 27 Haziran 2014 İst...",spotify:track:0CFgePbuBfvMvornA4z4H4,"Pentagram:Apokalips - Garajistanbul, 27 Hazira...",0CFgePbuBfvMvornA4z4H4
3,Hellbilly Deluxe,Rob Zombie,Dragula,spotify:track:6Nm8h73ycDG2saCnZV8poF,Rob Zombie:Dragula,6Nm8h73ycDG2saCnZV8poF
4,The Stage,Avenged Sevenfold,Runaway,spotify:track:071hmHI51coELRK8hoH12U,Avenged Sevenfold:Runaway,071hmHI51coELRK8hoH12U


In [ ]:
# create final dict as a copy df_stream
df_tableau = df_stream.copy()

# add column checking if streamed song is in library
# not used in this project but could be helpful for cool visualizations
df_tableau['In Library'] = np.where(df_tableau['UniqueID'].isin(df_library['UniqueID'].tolist()),1,0)

# left join with df_library on UniqueID to bring in album and track_uri
df_tableau = pd.merge(df_tableau, df_library[['Album','UniqueID','Track_Uri']],how='left',on=['UniqueID'])

df_tableau.head()

,endTime,artistName,trackName,msPlayed,UniqueID,In Library,Album,Track_Uri
0,2020-09-21 07:39,Erik Eriksson,White Noise 3 Hour Long,2926566,Erik Eriksson:White Noise 3 Hour Long,0,NaN,NaN
1,2020-09-21 15:03,Ini Kamoze,WORLD-A-REGGAE Music,35390,Ini Kamoze:WORLD-A-REGGAE Music,1,16 Vibes,0hr9CoeDSUMmQQ7ZgTLRzh
2,2020-09-21 17:40,Exodus,Salt the Wound,221866,Exodus:Salt the Wound,0,NaN,NaN
3,2020-09-22 06:32,Erik Eriksson,White Noise 3 Hour Long,9967424,Erik Eriksson:White Noise 3 Hour Long,0,NaN,NaN
4,2020-09-22 07:48,Megadeth,Dystopia,183490,Megadeth:Dystopia,1,Dystopia,0KiRc3f3OI9C7s0S1QjQNO


In [ ]:
# save your IDs from new project in Spotify Developer Dashboard
CLIENT_ID = 'a8c3529aca65472ea9058145146b25b5'
CLIENT_SECRET = '0b444e5ae8004cdfa702025ab2214b3a'

In [ ]:
# generate access token

# authentication URL
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [ ]:
# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [ ]:
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

In [ ]:
# create blank dictionary to store track URI, artist URI, and genres
dict_genre = {}

# convert track_uri column to an iterable list
track_uris = df_library['Track_Uri'].to_list()

# loop through track URIs and pull artist URI using the API,
# then use artist URI to pull genres associated with that artist
# store all these in a dictionary
for t_uri in track_uris:
    
    dict_genre[t_uri] = {'artist_uri': "", "genres":[]}
    
    r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)
    r = r.json()
    a_uri = r['artists'][0]['uri'].split(':')[2]
    dict_genre[t_uri]['artist_uri'] = a_uri
    
    s = requests.get(BASE_URL + 'artists/' + a_uri, headers=headers)
    s = s.json()
    dict_genre[t_uri]['genres'] = s['genres']

In [ ]:
# convert dictionary into dataframe with track_uri as the first column
df_genre = pd.DataFrame.from_dict(dict_genre, orient='index')
df_genre.insert(0, 'track_uri', df_genre.index)
df_genre.reset_index(inplace=True, drop=True)

df_genre.head()

,track_uri,artist_uri,genres
0,7FDuSqP0qxvFFrcftd93aB,2ye2Wgw4gimLv2eAKyk1NB,"[hard rock, metal, old school thrash, rock, th..."
1,5uKsIijGkAbWmYAEdnCPhG,5M52tdBnJaKSvOpJGz8mfZ,"[album rock, birmingham metal, classic rock, h..."
2,0CFgePbuBfvMvornA4z4H4,1Xz8iP9Dvl5uI88iraOhs7,"[oriental metal, turkish metal]"
3,6Nm8h73ycDG2saCnZV8poF,3HVdAiMNjYrQIKlOGxoGh5,"[alternative metal, hard rock, industrial rock..."
4,071hmHI51coELRK8hoH12U,0nmQIMXWTXfhgOBdNzhGOs,"[alternative metal, nu metal]"


In [ ]:
df_genre_expanded = df_genre.explode('genres')
df_genre_expanded.head()

,track_uri,artist_uri,genres
0,7FDuSqP0qxvFFrcftd93aB,2ye2Wgw4gimLv2eAKyk1NB,hard rock
0,7FDuSqP0qxvFFrcftd93aB,2ye2Wgw4gimLv2eAKyk1NB,metal
0,7FDuSqP0qxvFFrcftd93aB,2ye2Wgw4gimLv2eAKyk1NB,old school thrash
0,7FDuSqP0qxvFFrcftd93aB,2ye2Wgw4gimLv2eAKyk1NB,rock
0,7FDuSqP0qxvFFrcftd93aB,2ye2Wgw4gimLv2eAKyk1NB,thrash metal


In [ ]:
# save df_tableau and df_genre_expanded as csv files that we can load into Tableau
df_tableau.to_csv('MySpotifyDataTable.csv')
df_genre_expanded.to_csv('GenresExpandedTable.csv')

print('done')

done


Until this part, I prepared and export the data for the usage in Tableau. After this, I used Tableau Public program to make some visualizations